### Ticket DBP-443
Contributors - *Akhila Godeshi, Chervirala Sai Pranathi, Naveen Jujaray*
### Description

1. Convert the flags column values to Y or N when it has the value as below. the Flags (0 - N, NO - N, and 1 - Y, Yes-Y)

2. If it is blank then blank only. flags (INQ, PRO, ADM, CON, CAN, ENR, APP) and based on its date's column, the condition to be checked and fill the flags.

3. No Special characters, Numerics, data shifts are allowed.

It is applicable for the Below purposes.

    EM Marketing

    EM Search Vendor data

    EM GO

    EM Suppression

    EM Scoring

In [0]:
from itertools import chain
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, when
from pyspark.sql import functions as F
import warnings
from datetime import datetime

# Suppress warnings
warnings.filterwarnings("ignore")

# Define the schema for the DataFrame
schema = StructType([
    StructField("FLAG_PRO", StringType(), True),
    StructField("DATE_PRO", StringType(), True),
    StructField("FLAG_PUSH", StringType(), True),
    StructField("DATE_PUSH", StringType(), True),
    StructField("FLAG_INQ", StringType(), True),
    StructField("DATE_INQ", StringType(), True),
    StructField("FLAG_APP", StringType(), True),
    StructField("DATE_APP", StringType(), True),
    StructField("FLAG_ADM", StringType(), True),
    StructField("DATE_ADM", StringType(), True),
    StructField("FLAG_CON", StringType(), True),
    StructField("DATE_CON", StringType(), True),
    StructField("FLAG_ENR", StringType(), True),
    StructField("DATE_ENR", StringType(), True),
    StructField("FLAG_CANC", StringType(), True),
    StructField("DATE_CANC", StringType(), True),
    StructField("FLAG_INTERNATIONAL", StringType(), True),
    StructField("DATE_INTERNATIONAL", StringType(), True),
    StructField("FLAG_WAIT", StringType(), True),
    StructField("DATE_WAIT", StringType(), True),
    StructField("FLAG_DEFER", StringType(), True),
    StructField("DATE_DEFER", StringType(), True),   
    StructField("FLAG_ETHNIC", StringType(), True),
    StructField("DATE_ETHNIC", StringType(), True)    
])

# Define the data with random values
data = [
    ("1", "05/12/2022", "Jai Ballaya", "01/25/2022", "One", "01/01/2023", "No", "01/02/2023", "Y", "01/03/2023", "0", "01/04/2023", "No", "01/05/2023", "Y", "01/06/2023", "0", "", "Y", "01/06/2023", "0", "", "N", "01/09/2023"),
    ("0", "05/12/2022", "No", "", "", "01/11/2023", "Yes", "01/12/2023", "N", "01/13/2023", "1", "01/14/2023", "No", "01/15/2023", "Y", "01/16/2023", "0", "", "N", "01/06/2023", "1", "01/18/2023", "No", "01/19/2023"),
    ("One", "05/12/2022", "Yes", "", "No", "01/21/2023", "No", "01/22/2023", "", "", "Zero", "", "No", "", "yippie", "", "0", "", "", "01/06/2023", "", "01/06/2023", "", "01/06/2023"),
    ("0", "05/12/2022", "Hurray", "01/06/2023", "Yes", "01/23/2023", "N", "01/24/2023", "1", "01/25/2023", "Yes", "01/26/2023", "N", "01/27/2023", "1", "01/28/2023", "No", "", "Y", "", "Yes", "", "", ""),
    ("1", "05/12/2022", "Yes", "01/06/2023", "No", "01/30/2023", "Y", "01/31/2023", "Naruto", "", "No", "", "Y", "", "0", "", "Yes", "", "N", "", "1", "", "", "")
]

# Create the DataFrame
df = spark.createDataFrame(data, schema)

# Display the DataFrame
df.display()

FLAG_PRO,DATE_PRO,FLAG_PUSH,DATE_PUSH,FLAG_INQ,DATE_INQ,FLAG_APP,DATE_APP,FLAG_ADM,DATE_ADM,FLAG_CON,DATE_CON,FLAG_ENR,DATE_ENR,FLAG_CANC,DATE_CANC,FLAG_INTERNATIONAL,DATE_INTERNATIONAL,FLAG_WAIT,DATE_WAIT,FLAG_DEFER,DATE_DEFER,FLAG_ETHNIC,DATE_ETHNIC
1,05/12/2022,Jai Ballaya,01/25/2022,One,01/01/2023,No,01/02/2023,Y,01/03/2023,0,01/04/2023,No,01/05/2023,Y,01/06/2023,0,,Y,01/06/2023,0,,N,01/09/2023
0,05/12/2022,No,,,01/11/2023,Yes,01/12/2023,N,01/13/2023,1,01/14/2023,No,01/15/2023,Y,01/16/2023,0,,N,01/06/2023,1,01/18/2023,No,01/19/2023
One,05/12/2022,Yes,,No,01/21/2023,No,01/22/2023,,,Zero,,No,,yippie,,0,,,01/06/2023,,01/06/2023,,01/06/2023
0,05/12/2022,Hurray,01/06/2023,Yes,01/23/2023,N,01/24/2023,1,01/25/2023,Yes,01/26/2023,N,01/27/2023,1,01/28/2023,No,,Y,,Yes,,,
1,05/12/2022,Yes,01/06/2023,No,01/30/2023,Y,01/31/2023,Naruto,,No,,Y,,0,,Yes,,N,,1,,,


In [0]:
# Dictionary of required key mapping
conversion_mapping = {
    "1": "Y",
    "Yes": "Y",
    "Y": "Y",
    "0": "N",
    "No": "N",
    "N": "N"
}

# Specific Columns to be selected for the ticket
Flag_Columns = ['FLAG_PRO', 'FLAG_PUSH', 'FLAG_INQ', 'FLAG_APP', 'FLAG_ADM', 'FLAG_CON', 'FLAG_ENR', 'FLAG_CANC', 'FLAG_INTERNATIONAL', 'FLAG_WAIT', 'FLAG_DEFER', 'FLAG_DENY', 'FLAG_ETHNIC']

# Loop through each column in Flag_Columns
for column in Flag_Columns:
    # Check if column exists in the DataFrame
    if column in df.columns:
        # Apply the mapping using when and otherwise functions
        df = df.withColumn(column, when(col(column).isin(list(conversion_mapping.keys())), \
                                       F.create_map([F.lit(x) for x in chain(*conversion_mapping.items())]).getItem(col(column))) \
                                    .otherwise(None))
    else:
        # Create a new column with null values
        df = df.withColumn(column, F.lit(None))
### Made Flag Colums as Y if date is populated otherwise keep as it is
DateColumns=["DATE_PRO","DATE_PUSH","DATE_INQ","DATE_APP","DATE_ADM","DATE_CON","DATE_ENR","DATE_CANC","DATE_DENY","DATE_DEFER","DATE_WAIT","DATE_ETHNIC"]

# Iterate over the list of DateColumns
for i in DateColumns:
    # Check if the column 'i' is present in the DataFrame 'df'
    if i in df.columns:
        # Generate a new column name by prefixing "FLAG_" to the substring of 'i' starting from index 5
        j = "FLAG_" + i[5:]
        
        # Update the DataFrame 'df' with a new column 'j' using the given conditions
        df = df.withColumn(j, when((((col(i).isNotNull()) & (col(i) != "")) & ((col(j).isNull()) | (col(j) == "N") | (col(j) == ""))), "Y").otherwise(col(j)))
        
# Display the DataFrame 'df'
df.display()

FLAG_PRO,DATE_PRO,FLAG_PUSH,DATE_PUSH,FLAG_INQ,DATE_INQ,FLAG_APP,DATE_APP,FLAG_ADM,DATE_ADM,FLAG_CON,DATE_CON,FLAG_ENR,DATE_ENR,FLAG_CANC,DATE_CANC,FLAG_INTERNATIONAL,DATE_INTERNATIONAL,FLAG_WAIT,DATE_WAIT,FLAG_DEFER,DATE_DEFER,FLAG_ETHNIC,DATE_ETHNIC,FLAG_DENY
Y,05/12/2022,Y,01/25/2022,Y,01/01/2023,Y,01/02/2023,Y,01/03/2023,Y,01/04/2023,Y,01/05/2023,Y,01/06/2023,N,,Y,01/06/2023,N,,Y,01/09/2023,null
Y,05/12/2022,N,,Y,01/11/2023,Y,01/12/2023,Y,01/13/2023,Y,01/14/2023,Y,01/15/2023,Y,01/16/2023,N,,Y,01/06/2023,Y,01/18/2023,Y,01/19/2023,null
Y,05/12/2022,Y,,Y,01/21/2023,Y,01/22/2023,null,,null,,N,,null,,N,,Y,01/06/2023,Y,01/06/2023,Y,01/06/2023,null
Y,05/12/2022,Y,01/06/2023,Y,01/23/2023,Y,01/24/2023,Y,01/25/2023,Y,01/26/2023,Y,01/27/2023,Y,01/28/2023,N,,Y,,Y,,null,,null
Y,05/12/2022,Y,01/06/2023,Y,01/30/2023,Y,01/31/2023,null,,N,,Y,,N,,Y,,N,,Y,,null,,null
